In [12]:
pwd

'C:\\Users\\Anthony.Sledge\\Desktop\\Code\\data_wrangling_xml'

In [4]:
import pandas as pd
import numpy as np

In [5]:
from xml.etree import ElementTree as ET

In [6]:
doc = ET.parse('./data/mondial_database.xml')

In [44]:
# Print names of all countries
for child in doc.getroot():
    print(child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra
France
Spain
Austria
Czech Republic
Germany
Hungary
Italy
Liechtenstein
Slovakia
Slovenia
Switzerland
Belarus
Latvia
Lithuania
Poland
Ukraine
Russia
Belgium
Luxembourg
Netherlands
Bosnia and Herzegovina
Croatia
Bulgaria
Romania
Turkey
Denmark
Estonia
Faroe Islands
Finland
Norway
Sweden
Monaco
Gibraltar
Guernsey
Holy See
Ceuta
Melilla
Iceland
Ireland
San Marino
Jersey
Malta
Isle of Man
Moldova
Portugal
Svalbard
United Kingdom
Afghanistan
China
Iran
Pakistan
Tajikistan
Turkmenistan
Uzbekistan
Armenia
Georgia
Azerbaijan
Bahrain
Bangladesh
Myanmar
India
Bhutan
Brunei
Malaysia
Laos
Thailand
Cambodia
Vietnam
Kazakhstan
North Korea
Kyrgyzstan
Hong Kong
Macao
Mongolia
Nepal
Christmas Island
Cocos Islands
Cyprus
Gaza Strip
Israel
Egypt
Indonesia
Timor-Leste
Papua New Guinea
Iraq
Jordan
Kuwait
Saudi Arabia
Syria
Lebanon
West Bank
Japan
South Korea
Maldives
Oman
United Arab Emirates
Yemen
Philippines
Qatar
Singapore
Sri Lanka
Taiwan
Anguil

In [43]:
for element in doc.iterfind('country'):
    print('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print(capitals_string[:-2])

* Albania:
Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:
Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:
Skopje, Kumanovo
* Serbia:
Beograd, Novi Sad, Niš
* Montenegro:
Podgorica
* Kosovo:
Prishtine
* Andorra:
Andorra la Vella
* France:
Strasbourg, Mulhouse, Bordeaux, Clermont-Ferrand, Caen, Rennes, Brest, Dijon, Orléans, Tours, Reims, Chalons en Champagne, Ajaccio, Besançon, Rouen, Le Havre, Paris, Boulogne-Billancourt, Argenteuil, Montreuil, Saint-Paul, Montpellier, Nîmes, Perpignan, Limoges, Metz, Nancy, Toulouse, Lille, Nantes, Angers, Le Mans, Amiens, Poitiers, Marseille, Nice, Toulon, Aix-en-Provence, Lyon, Grenoble, Saint-Étienne, Villeurbanne
* Spain:
Sevilla, Almería, Cádiz, Córdoba, Granada, Huelva, Málaga, Jerez de la Frontera, Jaén, Algeciras, Marbella, Dos Hermanas, Zaragoza, Oviedo, Gijón, Palma de 

In [9]:
#Read XML File
document = ET.parse('./data/mondial_database.xml')

<font size=4><b>1.) Find 10 countries with the lowest infant mortality rates</b> </font>

In [45]:
# Set blank dictionary
country_dict = {}

# Loop to find Country name and Infant Mortality rate
for element in document.iterfind('country'):
    country = element.find('name')
    infant = element.find('infant_mortality')

# Discard if there is no Country Name or Infant Mortality rate
    if (country) != None:
        if (infant) != None:
# Set Key as Country name and Value as Infant Mortality Rate
            country_dict[country.text] = float(infant.text)
        else:
            country_dict[country.text] = np.NaN

# Convert to data frame from dictionary
country_df = pd.DataFrame.from_dict(country_dict, orient='index')

# Set Column to Infant Mortality Rate
country_df.columns = ['Infant_Mortality_Rate']
country_df.sort_values(by='Infant_Mortality_Rate', ascending=True).head(10)

,Infant_Mortality_Rate
Monaco,1.81
Japan,2.13
Bermuda,2.48
Norway,2.48
Singapore,2.53
Sweden,2.60
Czech Republic,2.63
Hong Kong,2.73
Macao,3.13
Iceland,3.15


<font size=4><b>2.) Find the 10 cities with the largest population</b> </font>

In [86]:
# Create blank list for dataframe
list_for_df = []

# Loop to find city names and provinces
for country in document.iterfind('country'):
    ctry_cities = country.findall('city')
    provinces = country.findall('province')

# Some countries have province populations; this loop combines them
    for province in provinces:
        ctry_cities += province.findall('city')

# Loops for city name and last population entry
    for city in ctry_cities:
        city_name = city.find('name').text
        population = city.find('.//population[last()]')
# Skips popluation count if node is empty
        if population != None:
            city_pop = int(population.text)
# Gets the year that the population count is from
            recent_year = population.get('year')
        recent_pop = (country.find('name').text,
                      city_name, recent_year, city_pop)
        list_for_df.append(recent_pop)

# Creates dataframe
labels = ['Country', 'City', 'Year', 'Population']
city_df = pd.DataFrame.from_records(list_for_df, columns=labels)
city_df.sort_values('Population', ascending=False).head(10)

,Country,City,Year,Population
1341,China,Shanghai,2010,22315474
771,Turkey,Istanbul,2012,13710512
1527,India,Mumbai,2011,12442373
479,Russia,Moskva,2013,11979529
1340,China,Beijing,2010,11716620
2810,Brazil,São Paulo,2010,11152344
1342,China,Tianjin,2010,11090314
1064,China,Guangzhou,2010,11071424
1582,India,Delhi,2011,11034555
1067,China,Shenzhen,2010,10358381


<font size=4><b>3.) Find the 10 largest ethnic groups in the world </b> </font>

In [12]:
# Set blank dictionary
country_pop = {}

# Loops to find country name and recent population count
for element in document.iterfind('country'):
    country = element.find('name')
    population = element.find('.//population[last()]')

# Creates dictionary with Country as Key and Population count as Value
    if (country) != None:
        if (population) != None:
            country_pop[country.text] = int(population.text)
    else:
        country_pop[country.text] = np.NaN

# Creates dataframe
ctry_df = pd.DataFrame.from_dict(country_pop, orient='index')
ctry_df.reset_index(drop=False, inplace=True)
ctry_df.columns = ['Country', 'Population']

In [27]:
# Creates blank list to hold ethnicity percentages
ethnicity = []
ethnicity_labels = ['Country', 'Ethnicity', 'Percentage']

# Loops for ethnicgroup percentages and adds them to list
for country in document.iterfind('country'):
    ctry = country.find('name').text

    for ethnic in country.iterfind('ethnicgroup'):
        ethnic_name = ethnic.text
        ethnic_pct = ethnic.attrib['percentage']
        ethnicity.append([ctry, ethnic_name, ethnic_pct])

# Creates data frame from list and merges it with country popluation dataframe
ethnicity_df = pd.DataFrame(ethnicity, columns=ethnicity_labels)
ethnicity_df = ethnicity_df.merge(
    ctry_df[['Country', 'Population']], on='Country')
ethnicity_df.head()

,Country,Ethnicity,Percentage,Population
0,Albania,Albanian,95,2800138
1,Albania,Greek,3,2800138
2,Greece,Greek,93,10816286
3,Macedonia,Macedonian,64.2,2059794
4,Macedonia,Albanian,25.2,2059794


In [40]:
# Creates a column with Global percentage of ethnicities by dividing by total population
ethnicity_df['Fractions'] = (ethnicity_df['Percentage'].astype(
    float) / 100) * ethnicity_df['Population']
ethnicity_df.head()

,Country,Ethnicity,Percentage,Population,Fractions
0,Albania,Albanian,95,2800138,2.660131e+06
1,Albania,Greek,3,2800138,8.400414e+04
2,Greece,Greek,93,10816286,1.005915e+07
3,Macedonia,Macedonian,64.2,2059794,1.322388e+06
4,Macedonia,Albanian,25.2,2059794,5.190681e+05


In [89]:
#Groups by ethnicity to display top 10
eth_group_df = ethnicity_df.groupby('Ethnicity').sum()
eth_group_df.sort_values(by='Fractions', ascending=False, inplace=True)
eth_group_df.head(10)

,Population,Fractions
Ethnicity,,
Han Chinese,1360720000,1.245059e+09
Indo-Aryan,1210854977,8.718156e+08
European,1157295639,4.948722e+08
African,975352746,3.183251e+08
Dravidian,1210854977,3.027137e+08
Mestizo,279743964,1.577344e+08
Bengali,149772364,1.467769e+08
Russian,322438406,1.318570e+08
Japanese,127298000,1.265342e+08


<font size=4><b>4.) Name and country of: <br>
a) Longest river <br> 
b) Largest lake and <br>
c) Airport at highest elevation</b> </font>

In [16]:
# Creates a blank dictionary
ctry_dict = {}

# Fills dictionary with country code ('car code') and country name
for code in document.iterfind('country'):
    ctry_dict[code.attrib['car_code']] = code.find('name').text

In [29]:
# Creates dataframe from dictionary
ctry_codes_df = pd.DataFrame.from_dict(ctry_dict, orient='index')
ctry_codes_df.reset_index(drop=False, inplace=True)
ctry_codes_df.columns = ['Country_Code', 'Country']
ctry_codes_df.head()

,Country_Code,Country
0,AL,Albania
1,GR,Greece
2,MK,Macedonia
3,SRB,Serbia
4,MNE,Montenegro


In [30]:
# Creates blank list
river_list = []

# Loops for rivers' names, lengths, and country codes
for rivers in document.iterfind('river'):
    river_name = rivers.find('name').text
    river_length_element = rivers.find('length')
    if river_length_element != None:
        river_length = river_length_element.text
# Rivers run though multiple countries; codes must be split after all are selected
        for river_code in rivers.attrib['country'].split():
            river_ctry_code = ctry_dict[river_code]
    river_list.append([river_ctry_code, river_code,
                       river_name, float(river_length)])
# print(river_list)

In [31]:
# Creates dataframe from list
Country_River_Labels = ['Country', 'Code', 'River_Name', 'River_Length']
rivers_df = pd.DataFrame.from_records(river_list, columns=Country_River_Labels)
rivers_df.head()

,Country,Code,River_Name,River_Length
0,Iceland,IS,Thjorsa,230.0
1,Iceland,IS,Joekulsa a Fjoellum,206.0
2,Norway,N,Glomma,604.0
3,Norway,N,Lagen,322.0
4,Sweden,S,Goetaaelv,93.0


In [20]:
# Creates blank list
lake_list = []
#Loops for lakes' names, areas, and country codes
for lakes in document.iterfind('lake'):
    lake_name = lakes.find('name').text
    lake_area_element = lakes.find('area')
    if lake_area_element != None:
        lake_area = lake_area_element.text
        for lake_code in lakes.attrib['country'].split():
            lake_ctry_code = ctry_dict[lake_code]
    lake_list.append([lake_ctry_code, lake_code, lake_name, float(lake_area)])
# print (lake_list)

In [32]:
# Creates dataframe from list
Country_Lake_Labels = ['Country', 'Code', 'Lake_Name', 'Lake_Area']
lakes_df = pd.DataFrame.from_records(lake_list, columns=Country_Lake_Labels)
lakes_df.head()

,Country,Code,Lake_Name,Lake_Area
0,Finland,SF,Inari,1040.0
1,Finland,SF,Oulujaervi,928.0
2,Finland,SF,Kallavesi,472.0
3,Finland,SF,Saimaa,4370.0
4,Finland,SF,Paeijaenne,1118.0


In [22]:
# Creates blank list
airport_list = []

# Loops for aiports' names, elevations, and country codes
for airport in document.iterfind('airport'):
    airport_name = airport.find('name').text
    airport_elev_element = airport.find('elevation')
    if airport_elev_element != None:
        airport_elev = airport_elev_element.text
        for airport_code in airport.attrib['country'].split():
            airport_ctry_code = ctry_dict[airport_code]
    airport_list.append([airport_ctry_code, airport_code,
                         airport_name, airport_elev])
# print (airport_list)

In [33]:
# Creates dataframe from list
Country_Airport_Labels = ['Country', 'Code', 'Airport_Name', 'Airport_Elev']
airport_text_df = pd.DataFrame.from_records(
    airport_list, columns=Country_Airport_Labels)
airport_df = airport_text_df.apply(pd.to_numeric, errors='ignore')
airport_df.head()

,Country,Code,Airport_Name,Airport_Elev
0,Afghanistan,AFG,Herat,977.0
1,Afghanistan,AFG,Kabul Intl,1792.0
2,Albania,AL,Tirana Rinas,38.0
3,Algeria,DZ,Cheikh Larbi Tebessi,811.0
4,Algeria,DZ,Batna Airport,822.0


<font size=4><b>4a) Longest river </font>

In [36]:
# Combines all three dataframe into one, keeping NaNs
ctry_airport_lakes_df = airport_df.merge(lakes_df, on='Country', how='outer')
ctry_land_df = ctry_airport_lakes_df.merge(
    rivers_df, on='Country', how='outer')

# Result for longest River
ctry_land_df.sort_values(by='River_Length', ascending=False).head(1)

,Country,Code_x,Airport_Name,Airport_Elev,Code_y,Lake_Name,Lake_Area,Code,River_Name,River_Length
8408,Peru,PE,Padre Aldamiz,201.0,PE,Lake Titicaca,8372.0,PE,Amazonas,6448.0


<font size=4><b>4b) Largest lake </font>

In [37]:
# Result for longest lake
ctry_land_df.sort_values(by='Lake_Area', ascending=False).head(1)

,Country,Code_x,Airport_Name,Airport_Elev,Code_y,Lake_Name,Lake_Area,Code,River_Name,River_Length
17740,Turkmenistan,TM,Ashgabat,211.0,TM,Caspian Sea,386400.0,NaN,NaN,NaN


<font size=4><b>4c) Airport at highest elevation</b> </font>

In [39]:
# Result for highest airport
ctry_land_df.sort_values(by='Airport_Elev', ascending=False).head(1)

,Country,Code_x,Airport_Name,Airport_Elev,Code_y,Lake_Name,Lake_Area,Code,River_Name,River_Length
385,Bolivia,BOL,El Alto Intl,4063.0,BOL,Licancabur Crater Lake,0.005,BOL,Rio Madeira,1450.0
